<a href="https://colab.research.google.com/github/HadiAskari/YouTube_Denoiser_Models/blob/main/YouTube_Denoising_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json
import numpy as np
import pickle 
from collections import defaultdict
import pickle 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.models import load_model
from sklearn.model_selection import train_test_split
import copy
import h5py
from collections import OrderedDict

In [ ]:
os.chdir('/content/drive/My Drive/YT/denoiser/data_20211212')

In [ ]:
with open('dataset_final_joint_cate_100_2_test.json', 'r') as f:
    
    read_item=json.load(f)

In [ ]:
!python --version

Python 3.7.12


In [ ]:
print(read_item['rand_base_0'].keys())

dict_keys(['viewed', 'initial_homepage', 'recommendation_trail', 'homepage', 'start_time', 'end_time', 'puppetId', 'video_seq', 'user_id', 'cate_dist'])


In [ ]:
sorted_read_item=dict(sorted(read_item.items()))

In [ ]:
print(sorted_read_item['rand_base_0'].keys())

dict_keys(['viewed', 'initial_homepage', 'recommendation_trail', 'homepage', 'start_time', 'end_time', 'puppetId', 'video_seq', 'user_id', 'cate_dist'])


In [ ]:
rl_unobfuscated_videos_raw={}
rl_obfuscated_videos_raw={}
rl_unobfuscated_recommendations_raw={}
rl_obfuscated_recommendations_raw={}

rand_unobfuscated_videos_raw={}
rand_obfuscated_videos_raw={}
rand_unobfuscated_recommendations_raw={}
rand_obfuscated_recommendations_raw={}

i=0

for key in sorted_read_item.keys():
  i=i+1
  if (i >= 0 and i < 1159):
    rand_unobfuscated_videos_raw[key]=sorted_read_item[key]['viewed']
    rand_unobfuscated_recommendations_raw[key]=sorted_read_item[key]['cate_dist']
  if (i >= 1159 and i < 2317):
    rand_obfuscated_videos_raw[key]=sorted_read_item[key]['viewed']
    rand_obfuscated_recommendations_raw[key]=sorted_read_item[key]['cate_dist']
  if (i >= 2317 and i < 3475):
    rl_unobfuscated_videos_raw[key]=sorted_read_item[key]['viewed']
    rl_unobfuscated_recommendations_raw[key]=sorted_read_item[key]['cate_dist']
  if (i >= 3475 and i < 4633):
    rl_obfuscated_videos_raw[key]=sorted_read_item[key]['viewed']
    rl_obfuscated_recommendations_raw[key]=sorted_read_item[key]['cate_dist']      
  

print(i)  

4632


In [ ]:
print(len(rl_obfuscated_recommendations_raw['rl_obfu_0']))

18


In [ ]:
with open('video_ids_final_joint_cate_100_2_test.json', 'r') as f:
    
    read_item1=json.load(f)

In [ ]:
print(read_item1['--1uHerc2Ns'])

130327


In [ ]:
print(rand_unobfuscated_videos_raw)
print(rand_obfuscated_videos_raw)

{'rand_base_0': ['gK3Gerx_JvQ', 'rFtfBqnVAIY', 'lL0Awp-uah4', 'MNWC4NMWfe8', 'VZwZ5-YWWIg', 'e26zZ83Oh6Y', 'cTfxennz3xU', 'PK-Tn6R8f0s', 'NruWYK-KA6w', 'T7XfKf46iZY', 'hnB8jLCznHI', 'GDwVXxYbLeM', 'etG-u27CrRw', 'B7YDcLP2DeY', '4yQkik5IHMY', 'Aq-Hq62B8os', 'kLP1s0IeIWw', 'fdyFGUh_Kpo', 'V4PPWYCqPKo', 'RcXSfVyb7aQ', '8Qt3EZ19s_Y', 'aYNaS6r4H4Q', 'URjAZwhZ334', 'B83nmCSwRuw', 'KHBe0jT6S3U', 'G-AAx1SR_YU', '8ssAzbZBhqs', 'lNGQmzob_ew', 'PkLaWIDobGE', 'QXg8IP697kA', '4ZmYkCwmQ7I', 'mau4ffsveZk', 'FrIfes1L7NI', 'cYGuWN1OhD8', 'ppH3n4cXkF4'], 'rand_base_1': ['saJt1RKLF0k', 'LV8wWhjTKRU', 'vN30emwcNS4', 'H2V4U218osU', 'D5-3Jpjz6do', 'qt_OkgSOrkU', '4JUb8Zw7ucw', 'a8jcNBVWJyE', 'PhbMMBYijNM', 'd5xyYI9tNV8', 'mmNrQ7Smung', 'CG61rtebA1E', 'V1bFr2SWP1I', 'YrLk4vdY28Q', '2zjLBWnZGTU', 'aLOnQmmmlkw', 'oGt8Lnhgrz8', 'SjZMDqnmBsg', 'H9D2hr0WWh0', 'bLUBRShnBoo', 'N-H3FUTXlK8', 'C7hdUorDU-U', '9JblE5NyhEg', 'PKiAXVxOm6o', 'oybU09RQUTs', '7Zwrta7SjtM', 'XNamPomxdVM', '7HKKvfo5yMI', 'GI7p6fbHfx4', 'bUKnO

In [ ]:
for keys in rl_unobfuscated_videos_raw.keys():
  for i in range(len(rl_unobfuscated_videos_raw[keys])):
    code=rl_unobfuscated_videos_raw[keys][i]
    ID=read_item1[code]
    rl_unobfuscated_videos_raw[keys][i]=ID

In [ ]:
for keys in rl_obfuscated_videos_raw.keys():
  for i in range(len(rl_obfuscated_videos_raw[keys])):
    code=rl_obfuscated_videos_raw[keys][i]
    ID=read_item1[code]
    rl_obfuscated_videos_raw[keys][i]=ID

In [ ]:
for keys in rand_unobfuscated_videos_raw.keys():
  for i in range(len(rand_unobfuscated_videos_raw[keys])):
    code=rand_unobfuscated_videos_raw[keys][i]
    ID=read_item1[code]
    rand_unobfuscated_videos_raw[keys][i]=ID

In [ ]:
for keys in rand_obfuscated_videos_raw.keys():
  for i in range(len(rand_obfuscated_videos_raw[keys])):
    code=rand_obfuscated_videos_raw[keys][i]
    ID=read_item1[code]
    rand_obfuscated_videos_raw[keys][i]=ID

In [ ]:
print(rl_unobfuscated_videos_raw)
print(rl_obfuscated_videos_raw)

{'rl_base_0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], 'rl_base_1': [299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 24, 331, 332], 'rl_base_10': [571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 583, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604], 'rl_base_100': [865, 495, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 584, 895, 896, 897, 898, 899, 900, 901, 902], 'rl_base_1000': [1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1110, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 112, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132], 

In [ ]:
with h5py.File('video_embeddings_final_joint_cate_100_2_test_aug.hdf5', 'r') as f:
  emb=f['embeddings']
  for keys in rl_unobfuscated_videos_raw.keys():
    for i in range(len(rl_unobfuscated_videos_raw[keys])):
      code=rl_unobfuscated_videos_raw[keys][i]
      embed=emb[code]
      rl_unobfuscated_videos_raw[keys][i]=embed  
      
  for keys in rl_obfuscated_videos_raw.keys():
    for i in range(len(rl_obfuscated_videos_raw[keys])):
      code=rl_obfuscated_videos_raw[keys][i]
      embed=emb[code]
      rl_obfuscated_videos_raw[keys][i]=embed   

  for keys in rand_unobfuscated_videos_raw.keys():
    for i in range(len(rand_unobfuscated_videos_raw[keys])):
      code=rand_unobfuscated_videos_raw[keys][i]
      embed=emb[code]
      rand_unobfuscated_videos_raw[keys][i]=embed  
  
  for keys in rand_obfuscated_videos_raw.keys():
    for i in range(len(rand_obfuscated_videos_raw[keys])):
      code=rand_obfuscated_videos_raw[keys][i]
      embed=emb[code]
      rand_obfuscated_videos_raw[keys][i]=embed          

In [ ]:
print(np.shape(rand_unobfuscated_videos_raw['rand_base_1158']))

(40, 403)


In [ ]:
lst=[]
for keys in rl_unobfuscated_videos_raw.keys():
  lst.append(rl_unobfuscated_videos_raw[keys])

In [ ]:
print(np.shape(lst[1000]))

(40, 403)


In [ ]:
count=0

zero=np.zeros(403)

print(zero.shape)


for i in range(len(lst)):
  if len(lst[i]) != 40:
   while True:
      lst[i].append(zero)
      if len(lst[i]) == 40:
        break
    


(403,)


In [ ]:
I = np.array(lst)

In [ ]:
print(I.shape)

(1158, 40, 403)


In [ ]:
lst1=[]
for keys in rl_obfuscated_videos_raw.keys():
  lst1.append(rl_obfuscated_videos_raw[keys])

In [ ]:
for i in range(len(lst1)):
  if len(lst1[i]) != 50:
    while True:
      lst1[i].append(zero)
      if len(lst1[i]) == 50:
        break
   


In [ ]:
Iprime=np.array(lst1)

In [ ]:
print(Iprime.shape)

(1158, 50, 403)


In [ ]:
lst=[]
for keys in rand_unobfuscated_videos_raw.keys():
  lst.append(rand_unobfuscated_videos_raw[keys])

In [ ]:
print(np.shape(lst[1000]))

(40, 403)


In [ ]:
count=0

zero=np.zeros(403)

print(zero.shape)


for i in range(len(lst)):
  if len(lst[i]) != 40:
   while True:
      lst[i].append(zero)
      if len(lst[i]) == 40:
        break

(403,)


In [ ]:
Irand = np.array(lst)

In [ ]:
print(Irand.shape)

(1158, 40, 403)


In [ ]:
lst1=[]
for keys in rand_obfuscated_videos_raw.keys():
  lst1.append(rand_obfuscated_videos_raw[keys])

In [ ]:
for i in range(len(lst1)):
  if len(lst1[i]) != 50:
    while True:
      lst1[i].append(zero)
      if len(lst1[i]) == 50:
        break

In [ ]:
Irand_prime=np.array(lst1)

In [ ]:
print(Irand_prime.shape)

(1158, 50, 403)


In [ ]:
llist=[]

for key in rl_unobfuscated_recommendations_raw.keys():
  llist.append(rl_unobfuscated_recommendations_raw[key])


In [ ]:
O = np.array(llist)

In [ ]:
print(O.shape)

(1158, 18)


In [ ]:
llist1=[]

for key in rl_obfuscated_recommendations_raw.keys():
  llist1.append(rl_obfuscated_recommendations_raw[key])

In [ ]:
Oprime = np.array(llist1)

In [ ]:
print(Oprime.shape)

(1158, 18)


In [ ]:
llist=[]

for key in rand_unobfuscated_recommendations_raw.keys():
  llist.append(rand_unobfuscated_recommendations_raw[key])


In [ ]:
Orand = np.array(llist)

In [ ]:
print(O.shape)

(1158, 18)


In [ ]:
llist1=[]

for key in rand_obfuscated_recommendations_raw.keys():
  llist1.append(rand_obfuscated_recommendations_raw[key])

In [ ]:
Orand_prime = np.array(llist1)

In [ ]:
print(Oprime.shape)

(1158, 18)


In [ ]:
I_train, I_test, Iprime_train, Iprime_test, O_train, O_test, Oprime_train, Oprime_test = train_test_split(I, Iprime, O, Oprime, test_size=0.2, random_state=42)

In [ ]:
print(I_train.shape)
print(Iprime_train.shape)

(926, 40, 403)
(926, 50, 403)


In [ ]:
I_test = np.expand_dims(I_test, axis=3)
Iprime_test= np.expand_dims(Iprime_test, axis=3)

In [ ]:
I_train = np.expand_dims(I_train, axis=3)
Iprime_train= np.expand_dims(Iprime_train, axis=3)
I_train.shape

(926, 40, 403, 1)

In [ ]:
input_I=Input(
    shape=(40,403,1), name="Input_I")

input_Iprime=Input(
    shape=(50,403,1), name="Input_Iprime")

input_Oprime=Input(
    shape=(18), name="Input_Oprime")

In [ ]:
I_CNN_1=Conv2D(1, (3, 3), padding="same", activation= 'relu', name='I_CNN_1', input_shape = (40,403,1))(input_I)
Iprime_CNN_1=Conv2D(1, (3, 3), padding="same",activation= 'relu', name='Iprime_CNN_1', input_shape = (50,403,1))(input_Iprime)
Oprime_FC_1=Dense(18,activation= 'relu', name='Oprime_FC_1')(input_Oprime)
max_pool_I_CNN_1=MaxPooling2D(pool_size=(2,4), padding='valid')(I_CNN_1)
max_pool_I_CNN_2=MaxPooling2D(pool_size=(2, 4),  padding='valid')(Iprime_CNN_1)


In [ ]:
concat=concatenate([Flatten()(max_pool_I_CNN_1), Flatten()(max_pool_I_CNN_2), Flatten()(Oprime_FC_1)])

In [ ]:
concat=Dropout(0.2)(concat)

In [ ]:
output_layer=Dense(18, activation= 'softmax', name='Output_layer')(concat)

In [ ]:
model=keras.Model(inputs=[input_I, input_Iprime, input_Oprime], outputs=output_layer)

In [ ]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_I (InputLayer)           [(None, 40, 403, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Input_Iprime (InputLayer)      [(None, 50, 403, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 I_CNN_1 (Conv2D)               (None, 40, 403, 1)   10          ['Input_I[0][0]']                
                                                                                            

In [ ]:
model.compile(optimizer='adam', loss= tf.keras.losses.KLDivergence(), metrics=[tf.keras.losses.KLDivergence()])

In [ ]:
model.fit(
    [I_train, Iprime_train, Oprime_train],
    [O_train],
    batch_size=16,
    epochs=300)

Epoch 1/300
58/58 [==============================] - 1s 8ms/step - loss: 0.5013 - kl_divergence: 0.5013
Epoch 2/300
58/58 [==============================] - 0s 8ms/step - loss: 0.4271 - kl_divergence: 0.4271
Epoch 3/300
58/58 [==============================] - 0s 8ms/step - loss: 0.3837 - kl_divergence: 0.3837
Epoch 4/300
58/58 [==============================] - 0s 8ms/step - loss: 0.3441 - kl_divergence: 0.3440
Epoch 5/300
58/58 [==============================] - 0s 8ms/step - loss: 0.3136 - kl_divergence: 0.3136
Epoch 6/300
58/58 [==============================] - 0s 8ms/step - loss: 0.2913 - kl_divergence: 0.2913
Epoch 7/300
58/58 [==============================] - 0s 8ms/step - loss: 0.2647 - kl_divergence: 0.2648
Epoch 8/300
58/58 [==============================] - 0s 8ms/step - loss: 0.2423 - kl_divergence: 0.2423
Epoch 9/300
58/58 [==============================] - 0s 8ms/step - loss: 0.2263 - kl_divergence: 0.2263
Epoch 10/300
58/58 [==============================] - 0s 8ms/ste

In [ ]:
model.evaluate([I_test, Iprime_test, Oprime_test],
    [O_test],
    batch_size=16)

15/15 [==============================] - 0s 5ms/step - loss: 0.3637 - kl_divergence: 0.3627


[0.3636927902698517, 0.36267417669296265]

In [ ]:
#Testing for I only

In [ ]:
input_surrogate=Input(
    shape=(40,403,1), name="Input_surrogate")

In [ ]:
I_CNN_1=Conv2D(1, (3, 3), padding="same", activation= 'relu', name='I_CNN_1', input_shape = (40,403,1))(input_surrogate)

In [ ]:
max_pool_I_CNN_1=MaxPooling2D(pool_size=(2,4), padding='valid')(I_CNN_1)


In [ ]:
concat=concatenate([Flatten()(max_pool_I_CNN_1)])

In [ ]:
concat=Dropout(0.2)(concat)

In [ ]:
output_layer=Dense(18, activation= 'softmax', name='Output_layer')(concat)

In [ ]:
model=keras.Model(inputs=[input_surrogate], outputs=output_layer)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_surrogate (InputLayer  [(None, 40, 403, 1)]     0         
 )                                                               
                                                                 
 I_CNN_1 (Conv2D)            (None, 40, 403, 1)        10        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 20, 100, 1)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2000)              0         
                                                                 
 concatenate (Concatenate)   (None, 2000)              0         
                                                                 
 dropout (Dropout)           (None, 2000)              0     

In [ ]:
model.compile(optimizer='adam', loss= tf.keras.losses.KLDivergence(), metrics=[tf.keras.losses.KLDivergence()])

In [ ]:
model.fit(
    [I_train],
    [O_train],
    batch_size=16,
    epochs=300)

Epoch 1/300
58/58 [==============================] - 10s 6ms/step - loss: 0.5072 - kl_divergence: 0.5071
Epoch 2/300
58/58 [==============================] - 0s 6ms/step - loss: 0.4475 - kl_divergence: 0.4474
Epoch 3/300
58/58 [==============================] - 0s 5ms/step - loss: 0.4227 - kl_divergence: 0.4228
Epoch 4/300
58/58 [==============================] - 0s 6ms/step - loss: 0.4017 - kl_divergence: 0.4016
Epoch 5/300
58/58 [==============================] - 0s 6ms/step - loss: 0.3806 - kl_divergence: 0.3806
Epoch 6/300
58/58 [==============================] - 0s 6ms/step - loss: 0.3604 - kl_divergence: 0.3602
Epoch 7/300
58/58 [==============================] - 0s 6ms/step - loss: 0.3427 - kl_divergence: 0.3428
Epoch 8/300
58/58 [==============================] - 0s 6ms/step - loss: 0.3274 - kl_divergence: 0.3275
Epoch 9/300
58/58 [==============================] - 0s 6ms/step - loss: 0.3177 - kl_divergence: 0.3179
Epoch 10/300
58/58 [==============================] - 0s 6ms/st

In [ ]:
model.evaluate([I_test],
    [O_test],
    batch_size=16)

15/15 [==============================] - 0s 5ms/step - loss: 0.4572 - kl_divergence: 0.4564


[0.4572131633758545, 0.4563773572444916]

In [ ]:
#rand testing

In [ ]:
I_train, I_test, Iprime_train, Iprime_test, O_train, O_test, Oprime_train, Oprime_test = train_test_split(Irand, Irand_prime, Orand, Orand_prime, test_size=0.2, random_state=42)